In [115]:
import pandas as pd
import datetime as dt
import urllib
import zipfile
import re
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from matplotlib.dates import HourLocator

## download GTFS data

In [39]:
url = 'http://web.mta.info/developers/data/nyct/subway/google_transit.zip'
urllib.urlretrieve (url, "GTFS_nyc_Subway.zip")

('GTFS_nyc_Subway.zip', <httplib.HTTPMessage instance at 0x131284950>)

In [40]:
zip_ref = zipfile.ZipFile("GTFS_nyc_Subway.zip", 'r')
zip_ref.extractall("GTFS_nyc_Subway")
zip_ref.close()

In [41]:
stop_times= pd.read_csv('GTFS_nyc_Subway/stop_times.txt')

In [42]:
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,A20171105SAT_036000_GS.N01R,06:00:00,06:00:00,901N,1,NaN,0,0,NaN
1,A20171105SAT_036000_GS.N01R,06:01:30,06:01:30,902N,2,NaN,0,0,NaN
2,A20171105SAT_036400_GS.S01R,06:04:00,06:04:00,902S,1,NaN,0,0,NaN
3,A20171105SAT_036400_GS.S01R,06:05:30,06:05:30,901S,2,NaN,0,0,NaN
4,A20171105SAT_037000_GS.N01R,06:10:00,06:10:00,901N,1,NaN,0,0,NaN


In [43]:
stop_times.shape

(554831, 9)

Interpreting trip_id:
- A20171105SAT refers to service by Saturday schedule
- 036000 refers to trip id
- GS.N01R refers to line and North bound, and the set of stops it'll stop at

In [44]:
stop_times['service_id']=stop_times['trip_id'].apply(lambda x: x.split('_')[0])
stop_times['sub_trip_id']=stop_times['trip_id'].apply(lambda x: x.split('_')[1])
stop_times['train+direction']=stop_times['trip_id'].apply(lambda x: x.split('_')[2])
del stop_times['stop_headsign']
del stop_times['pickup_type']
del stop_times['drop_off_type']
del stop_times['shape_dist_traveled']

In [45]:
stop_times['train'] = stop_times['train+direction'].apply(lambda x: x.split('.')[0])
stop_times['day'] = stop_times['service_id'].apply(lambda x: x[-3:])

In [63]:
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,service_id,sub_trip_id,train+direction,train,day
0,A20171105SAT_036000_GS.N01R,1900-01-01 06:00:00,06:00:00,901N,1,A20171105SAT,036000,GS.N01R,GS,SAT
1,A20171105SAT_036000_GS.N01R,1900-01-01 06:01:30,06:01:30,902N,2,A20171105SAT,036000,GS.N01R,GS,SAT
2,A20171105SAT_036400_GS.S01R,1900-01-01 06:04:00,06:04:00,902S,1,A20171105SAT,036400,GS.S01R,GS,SAT
3,A20171105SAT_036400_GS.S01R,1900-01-01 06:05:30,06:05:30,901S,2,A20171105SAT,036400,GS.S01R,GS,SAT
4,A20171105SAT_037000_GS.N01R,1900-01-01 06:10:00,06:10:00,901N,1,A20171105SAT,037000,GS.N01R,GS,SAT


In [47]:
#drop all records with time > 24:00:00
stop_times = stop_times.drop(stop_times[(stop_times.arrival_time > '23:59:59')].index)

In [65]:
stop_times['arrival_time'] =  pd.to_datetime(stop_times['arrival_time'], format='%H:%M').dt.time

In [67]:
stop_times.dtypes

trip_id            object
arrival_time       object
departure_time     object
stop_id            object
stop_sequence       int64
service_id         object
sub_trip_id        object
train+direction    object
train              object
day                object
dtype: object

In [70]:
stop_times.shape

(536886, 10)

In [71]:
stop_times['trip_id'].nunique()

20351

In [72]:
stop_times['train'].unique()

array(['GS', '1', '2', '3', '4', '5', '6', '7', 'N', 'Q', 'R', 'A', 'B',
       'C', 'D', 'E', 'F', 'FS', 'G', 'H', 'J', 'L', 'M', 'SI'], dtype=object)

In [73]:
stop_times['day'].unique()

array(['SAT', 'SUN', 'WKD'], dtype=object)

In [74]:
stop_times.loc[stop_times[stop_times['train+direction'].isin(['N..N72R','N..S72R'])].index,['train']]='W'

In [75]:
stop_times.to_csv('stop_times_cleaned')

No Z train info in schedule data. Z is same as J.

FS(Brooklyn), GS(Time sq- Grand Central), H(JFK) are all S train in map, 

In [76]:
stops=pd.read_csv('GTFS_nyc_Subway/stops.txt')

In [77]:
stop_times = stop_times.merge(stops[['stop_id','stop_name','stop_lat','stop_lon']],on='stop_id',how='left')

In [78]:
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,service_id,sub_trip_id,train+direction,train,day,stop_name,stop_lat,stop_lon
0,A20171105SAT_036000_GS.N01R,06:00:00,06:00:00,901N,1,A20171105SAT,036000,GS.N01R,GS,SAT,Grand Central - 42 St,40.752769,-73.979189
1,A20171105SAT_036000_GS.N01R,06:01:30,06:01:30,902N,2,A20171105SAT,036000,GS.N01R,GS,SAT,Times Sq - 42 St,40.755983,-73.986229
2,A20171105SAT_036400_GS.S01R,06:04:00,06:04:00,902S,1,A20171105SAT,036400,GS.S01R,GS,SAT,Times Sq - 42 St,40.755983,-73.986229
3,A20171105SAT_036400_GS.S01R,06:05:30,06:05:30,901S,2,A20171105SAT,036400,GS.S01R,GS,SAT,Grand Central - 42 St,40.752769,-73.979189
4,A20171105SAT_037000_GS.N01R,06:10:00,06:10:00,901N,1,A20171105SAT,037000,GS.N01R,GS,SAT,Grand Central - 42 St,40.752769,-73.979189


In [81]:
stop_times.shape

(536886, 13)

In [82]:
stop_times['arrival_time'] = stop_times['arrival_time'].astype(datetime)

In [83]:
stop_times.dtypes

trip_id             object
arrival_time        object
departure_time      object
stop_id             object
stop_sequence        int64
service_id          object
sub_trip_id         object
train+direction     object
train               object
day                 object
stop_name           object
stop_lat           float64
stop_lon           float64
dtype: object

## Visualize by lines

In [84]:
dict_of_regions = {k: v for k, v in stop_times.groupby('train')}

In [85]:
#for line 1
dict_of_regions['1'].head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,service_id,sub_trip_id,train+direction,train,day,stop_name,stop_lat,stop_lon
1356,A20171105WKD_029750_1..S03R,04:57:30,04:57:30,101S,1,A20171105WKD,029750,1..S03R,1,WKD,Van Cortlandt Park - 242 St,40.889248,-73.898583
1357,A20171105WKD_029750_1..S03R,04:59:00,04:59:00,103S,2,A20171105WKD,029750,1..S03R,1,WKD,238 St,40.884667,-73.900870
1358,A20171105WKD_029750_1..S03R,05:00:30,05:00:30,104S,3,A20171105WKD,029750,1..S03R,1,WKD,231 St,40.878856,-73.904834
1359,A20171105WKD_029750_1..S03R,05:02:00,05:02:00,106S,4,A20171105WKD,029750,1..S03R,1,WKD,Marble Hill - 225 St,40.874561,-73.909831
1360,A20171105WKD_029750_1..S03R,05:03:30,05:03:30,107S,5,A20171105WKD,029750,1..S03R,1,WKD,215 St,40.869444,-73.915279


In [86]:
dict_of_regions['1']['arrival_time'].unique()

array([datetime.time(4, 57, 30), datetime.time(4, 59),
       datetime.time(5, 0, 30), ..., datetime.time(4, 33),
       datetime.time(5, 13), datetime.time(5, 19, 30)], dtype=object)

In [89]:
dict_of_regions['1']['train+direction'].unique()

array(['1..S03R', '1..N03R', '1..S04R', '1..N12R', '1..S12R', '1..N13R'], dtype=object)

In [94]:
dict_of_regions['1']['sub_trip_id'].nunique()

873

In [129]:
#filter only northbound
north = ['1..N03R', '1..N12R', '1..N13R']
df1N = dict_of_regions['1'].loc[dict_of_regions['1']['train+direction'].isin(north)]


In [131]:
df1N.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,service_id,sub_trip_id,train+direction,train,day,stop_name,stop_lat,stop_lon
1394,A20171105WKD_036150_1..N03R,06:01:30,06:01:30,142N,1,A20171105WKD,036150,1..N03R,1,WKD,South Ferry,40.702068,-74.013664
1395,A20171105WKD_036150_1..N03R,06:04:00,06:04:00,139N,2,A20171105WKD,036150,1..N03R,1,WKD,Rector St,40.707513,-74.013783
1396,A20171105WKD_036150_1..N03R,06:05:00,06:05:00,138N,3,A20171105WKD,036150,1..N03R,1,WKD,Cortlandt St,40.711835,-74.012188
1397,A20171105WKD_036150_1..N03R,06:06:00,06:06:00,137N,4,A20171105WKD,036150,1..N03R,1,WKD,Chambers St,40.715478,-74.009266
1398,A20171105WKD_036150_1..N03R,06:07:30,06:07:30,136N,5,A20171105WKD,036150,1..N03R,1,WKD,Franklin St,40.719318,-74.006886
